<a href="https://colab.research.google.com/github/Etruscor/Python_Codes/blob/main/Price_notifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install pyshorteners
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import pyshorteners
from IPython.display import clear_output
import json
import re
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

MY_EMAIL = "etruscotest@gmail.com"
MY_PASSWORD = "etruscotest2022"
REPORT_CONTENT = {}

# url from Sheety api to read data in google sheets
retrive_sheets_base = "https://api.sheety.co/bc34554d8b63f6fdc0985e6dc9aae93d/baseImoveis/base"
retrive_sheets_empresa = "https://api.sheety.co/bc34554d8b63f6fdc0985e6dc9aae93d/baseImoveis/empresas"

# url from Sheety api to write data in google sheets
write_sheets_base = "https://api.sheety.co/bc34554d8b63f6fdc0985e6dc9aae93d/baseImoveis/base"
write_sheets_empresa = "https://api.sheety.co/bc34554d8b63f6fdc0985e6dc9aae93d/baseImoveis/empresas"

# web scraping site para o site www.vivareal.com.br
def get_data_vivareal(req_response,url_page):
  soup = BeautifulSoup(req_response.text, 'html.parser')
  prod_price =  re.sub('[^\x00-\x80]+', ' ', soup.find_all(name="span",class_=["price__list-value","rent-condominium","js-total-rental-price"])[1].text)
  if "Não informado" in prod_price:
     prod_price =  re.sub('[^\x00-\x80]+', ' ', soup.find_all(name="h3",class_=["price__price-info","js-price-sale"])[0].text)
  prod_adress =  re.sub('[^\x00-\x80]+', ' ', soup.find(name="p",class_="title__address").text)
  prod_link = short_url(url_page)
  #print(f'Preço: {prod_price}, End: {prod_adress}, Link: {prod_link}')
  return {'Prod_Price':prod_price,'Prod_Adress':prod_adress,'Link':prod_link}

# web scraping site para o site www.zapimoveis.com.br
def get_data_zapimoveis(response_dict,url_page):
  req = Request(response_dict['url'] , headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()
  soup = BeautifulSoup(webpage, 'html.parser')
  prod_price = re.sub('[^\x00-\x80]+', ' ', soup.find_all(name="span",class_=["price__value"])[2].text)
  prod_adress = re.sub('[^\x00-\x80]+', ' ', soup.find(name="span",class_="link").text)
  prod_link = short_url(url_page)
  return {'Prod_Price':prod_price,'Prod_Adress':prod_adress,'Link':prod_link}

# web scraping site www.olx.com.br
def get_data_olx(response_dict,url_page):
  req = Request(response_dict['url'] , headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()
  soup = BeautifulSoup(webpage, 'html.parser')
  prod_price =  re.sub('[^\x00-\x80]+', ' ', soup.find(name="h2",class_=["sc-ifAKCX","eQLrcK"]).text)
  prod_adress =  re.sub('[^\x00-\x80]+', ' ', soup.find(name="h1",class_=["sc-45jt43-0","eCghYu sc-ifAKCX cmFKIN"]).text)
  prod_link = short_url(url_page)
  return {'Prod_Price':prod_price,'Prod_Adress':prod_adress,'Link':prod_link}

# get a short version of a url
def short_url(long_url):
  type_tiny = pyshorteners.Shortener()
  short_url = type_tiny.tinyurl.short(long_url)
  return short_url

# check valid status code
def verify_status_code(req_response):
  code = req_response.status_code
  if code == 200 or 403:
    return True
  else:
    print(f'Status Code:{code}')
    return False

# check company name from EMPRESAS database in google sheets
def valid_site(company_name):
  empresas_response = requests.get(retrive_sheets_empresa)
  empresas_dict = empresas_response.json()
  for item in empresas_dict['empresas']:
    if company_name == item['nome']:
      return True
    else:
      return False

def generate_report():
  global REPORT_CONTENT
  # JSON data request from BASE database in google sheets as a dictinary
  db_response = requests.get(retrive_sheets_base)
  db_dict = db_response.json()
  counter = 0
  # iterate of each key 'base' from dictinary
  for counter,item in enumerate(db_dict['base']):
    ##counter += 1
    # url request from database dictionary
    response = requests.get(item['url'])
    # check if status code is valid
    if verify_status_code(response) == True:
      # get data price, adress and short url
      if 'VIVAREAL' == item['site']:
        product_data = get_data_vivareal(response,item['url'])
      elif 'ZAPIMOVEIS' == item['site']:
        product_data = get_data_zapimoveis(item,item['url'])
      elif 'OLX' == item['site']:
        product_data = get_data_olx(item,item['url'])
    else:
      print(f'''URL inválido! {item['url']}, code: {response.status_code}''')
    REPORT_CONTENT[counter] = product_data
  #print(REPORT_CONTENT)

def format_email_body(content):
  text = ""
  for i in content:
    text_input =  f"{content[i]['Prod_Price']}, {content[i]['Prod_Adress']}, {content[i]['Link']}<br>"
    text = text + text_input
  text = "Big Dick price list:<br><br>" + text
  return text

'''
running = True
while running:
  # Commands input
  print("Commands:\n1)Add - (Add New webpage to search)\n2)Report - (Get report)\n3)Exit")
  user_input = input("Entry your command: ")
  
  if user_input.upper() == 'REPORT':
    generate_report()
    user_choice = input("Continue: Y/N? ")
    if user_choice.upper() == 'N':
      clear_output() # clear prompt
      running = False
    else:
      running = True
      clear_output() # clear prompt
  elif user_input.upper() == 'ADD':
    url_input = input("Insira o URL do website a ser pesquisado (https//site.com.br/meu_ap): ")
    website_input = input("Digite o nome da empresa(OLX): ")
    if valid_site(website_input.upper()) == True:
      payload = {"base": {"id_site": "999","site": website_input.upper(),"url":url_input }} 
      requests.post(write_sheets_base,json=payload)
      print("Adicionado com sucesso!")
    else:
      print("Empresa não possuí suporte. Solicite ao Master Jedi pelo contato: alvarorj8@gmail.com")
  elif user_input.upper() == 'EXIT':
    running = False
  else:
    print('Comando inválido!')
'''
generate_report()
report_formated = format_email_body(REPORT_CONTENT)
# create message object instance
msg = MIMEMultipart()
 
message = report_formated

# add in the message body
msg.attach(MIMEText(message, 'html'))
 
# setup the parameters of the message
password = MY_PASSWORD
msg['From'] = MY_EMAIL
msg['To'] = "alvaro.azevedo@logictel.com.br"
msg['Subject'] = "Hugo-Man: No Way Home"
 
# create server
server = smtplib.SMTP('smtp.gmail.com: 587')
 
server.starttls()
 
# Login Credentials for sending the mail
server.login(msg['From'], password)
 
 
# send the message via the server.
server.sendmail(msg['From'], msg['To'], msg.as_string())
 
server.quit()

(221,
 b'2.0.0 closing connection k8-20020a67e3c8000000b0032d275e6913sm492755vsm.19 - gsmtp')